In [10]:
import torch
import torch.nn as nn

In [11]:
## Charger et modifier le modèle Deit pré-entraîné

model = torch.hub.load('facebookresearch/deit:main', 'deit_tiny_patch16_224', pretrained=True)

# Modifier la dernière couche pour avoir 2 sorties (tumeur vs non-tumeur)
in_features = model.head.in_features
model.head = nn.Linear(in_features, 2)

# Vérifier la nouvelle architecture
print(model)

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 192, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((192,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=192, out_features=576, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=192, out_features=192, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((192,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=192, out_features=768, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (fc2): Linear(in_features=768, out_features=192, bias=True)
        (drop2): Dropout(p=0.0, inplace=Fals

Using cache found in /Users/arthurgontier/.cache/torch/hub/facebookresearch_deit_main


In [17]:
import h5py
import numpy as np
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

def load_h5_array(file_path, key_default):
    with h5py.File(file_path, 'r') as f:
        # Si la clé par défaut existe, on l'utilise
        if key_default in f:
            return np.array(f[key_default])
        # Sinon, on prend la première clé disponible
        keys = list(f.keys())
        if len(keys) > 0:
            print(f"Clé '{key_default}' non trouvée dans {file_path}. Utilisation de la clé '{keys[0]}' à la place.")
            return np.array(f[keys[0]])
        else:
            raise KeyError(f"Aucune clé trouvée dans le fichier {file_path}")

class PCAMDatasetH5(Dataset):
    def __init__(self, images_path, labels_path, transform=None):
        self.transform = transform
        # On tente d'utiliser 'data' pour les images et 'labels' pour les labels
        self.images = load_h5_array(images_path, 'data')
        self.labels = load_h5_array(labels_path, 'labels')
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        img = self.images[idx]
        # Conversion en objet PIL pour appliquer les transformations
        img = Image.fromarray(img)
        label = int(self.labels[idx])
        if self.transform:
            img = self.transform(img)
        return img, label

# Définition des transformations : redimensionnement et normalisation
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # redimensionnement de 96x96 vers 224x224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Chemins vers les fichiers .h5 (adaptez ces chemins à votre organisation)
train_images_path = './data/camelyonpatch_level_2_split_valid_x.h5'
train_labels_path = './data/camelyonpatch_level_2_split_valid_y.h5'
test_images_path  = './data/camelyonpatch_level_2_split_test_x.h5'
test_labels_path  = './data/camelyonpatch_level_2_split_test_y.h5'

# Création des datasets
train_dataset = PCAMDatasetH5(train_images_path, train_labels_path, transform=transform)
test_dataset  = PCAMDatasetH5(test_images_path, test_labels_path, transform=transform)

print("Taille du jeu d'entraînement :", len(train_dataset))
print("Taille du jeu de test :", len(test_dataset))
print("Dimension d'une image d'entraînement :", train_dataset[0][0].shape)

# Création des DataLoaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print("Nombre de mini-batches dans l'entraînement :", len(train_loader))
print("Nombre de mini-batches dans le test :", len(test_loader))




Clé 'data' non trouvée dans ./data/camelyonpatch_level_2_split_valid_x.h5. Utilisation de la clé 'x' à la place.
Clé 'labels' non trouvée dans ./data/camelyonpatch_level_2_split_valid_y.h5. Utilisation de la clé 'y' à la place.
Clé 'data' non trouvée dans ./data/camelyonpatch_level_2_split_test_x.h5. Utilisation de la clé 'x' à la place.
Clé 'labels' non trouvée dans ./data/camelyonpatch_level_2_split_test_y.h5. Utilisation de la clé 'y' à la place.
Taille du jeu d'entraînement : 32768
Taille du jeu de test : 32768
Dimension d'une image d'entraînement : torch.Size([3, 224, 224])
Nombre de mini-batches dans l'entraînement : 512
Nombre de mini-batches dans le test : 512


/var/folders/6d/rj96yg4n5y336jbr308060rc0000gn/T/ipykernel_30468/2053369456.py:35: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  label = int(self.labels[idx])


In [18]:
## Training loop

lr = 1e-3
weight_decay = 1e-1

def train_model(model, dataloader, epochs):
  model.to(device)
  model.train()
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
  n_data = len(dataloader.dataset)
  losses = []
  accuracies = []
  epoch_times = []

  for epoch in range(epochs):
    running_loss, running_acc = 0., 0
    for inputs, targets in dataloader:
      inputs, targets = inputs.to(device), targets.to(device)
      outputs = model(inputs)
      loss = criterion(outputs, targets)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      preds = torch.argmax(outputs, dim=1)
      running_loss += loss.item()
      running_acc += torch.sum(preds == targets)
    print(f"Epoch {epoch+1}: Loss: {running_loss/n_data:.2f} Acc: {running_acc/n_data:.2f}")

    losses.append(running_loss/n_data)
    accuracies.append(running_acc/n_data)
    epoch_times.append(epoch+1)

  fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

  ax1.bar(epoch_times, losses, color="tab:red", alpha = 0.7)
  ax1.set_xlabel("Epochs")
  ax1.set_ylabel("Loss", color="tab:red")


  ax2.bar(epoch_times, [acc.cpu().numpy() for acc in accuracies], color="tab:blue", alpha=0.7)
  ax2.set_xlabel("Epochs")
  ax2.set_ylabel("Accuracy", color="tab:blue")

  fig.suptitle("Loss and Accuracy over Epochs")
  plt.show()


In [19]:
## Testing loop

def test_model(model, dataloader):
  model.to(device)
  model.eval()
  running_loss, running_acc = 0., 0
  n_data = len(dataloader.dataset)
  criterion = nn.CrossEntropyLoss()

  with torch.no_grad():

    for inputs, targets in dataloader:
      inputs, targets = inputs.to(device), targets.to(device)
      outputs = model(inputs)
      loss = criterion(outputs, targets)
      preds = torch.argmax(outputs,1)
      running_loss += loss.item()
      running_acc += torch.sum(preds == targets)


  loss_value = running_loss/n_data
  acc_value = running_acc/n_data

  fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

  ax1.bar("Loss", loss_value, color="tab:red", width=0.4, label="Loss")
  ax1.set_ylabel("Loss", color="tab:red")
  ax1.tick_params(axis="y", labelcolor="tab:red")


  ax2.bar("Accuracy", acc_value.cpu().numpy(), color="tab:blue", width=0.4, label="Accuracy")
  ax2.set_ylabel("Accuracy (%)", color="tab:blue")
  ax2.tick_params(axis="y", labelcolor="tab:blue")

  plt.title("Loss and Accuracy")
  plt.show()


In [21]:
# === Suite de fine-tuning du modèle DeiT (entraînement complet) ===

num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


train_model(model, train_loader, num_epochs)
test_model(model, test_loader)

# Sauvegarde du modèle fine-tuné
#torch.save(model.state_dict(), "deit_finetuned.pth")
#print("Modèle sauvegardé sous 'deit_finetuned.pth'")



/var/folders/6d/rj96yg4n5y336jbr308060rc0000gn/T/ipykernel_30468/2053369456.py:35: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  label = int(self.labels[idx])


KeyboardInterrupt: 